In [ ]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection("./")

In [ ]:
#This is to get a local refresh token for the local development
conn = openeo.connect("https://openeo.eodc.eu/openeo/1.2.0").authenticate_oidc()

The following cell shows the setup of 2 dummy processes linked together

since we are using the local processes dask we need to use the `datacube_from_process`

The params are mostly self explanatory however here are some notes:

The `input_stac_url` is still to be finalized as our usecase doesnt use it so its not fully tested if you want to use input data directly. Right now it gets converted into an environment variable `STAC_URL` which you can use in your process. so in theory right now you can use it in the `context` as well with the same result. In the future we may need it as a link to a preprocesses datacube from openeo.

leave `output` as output this is an OSCAR thing and since we return STAC collections and not files it is not used or meaningful in this case.

`oscar_scrip` is normally embedded in the `service_config` but if you want to overwrite it you can do that here.

That brings us to the configs:

`OSCAR/configs`

1. `oscar_script`: a bash script to run inside the container in the case of our dummy:
```bash
python script.py
```

2. `service_config` example:
```yaml
functions:
  oscar:
  - cluster_id:
      name: openeo-dummy
      cpu: '1.0'
      memory: 2Gi
      image: potato55/dummy-container:latest #change to -test if broken
      script: configs/dummy.sh # Path to the script from where the script is run
      log_level: CRITICAL   # To avoid supervisor logs in response
      vo: dev.intertwin.eu
      environment:
        variables:
          DUMMY_INPUT1: "dummy_value"
          DUMMY_INPUT2: "another_value"
          INPUT_STAC: "https://example.com/dummy-stac.json"
```

where you need to name your process, set resources, provide the docker image, and the script mentioned before. The environment variables can be set beforehand but they are updated using the `context` parameter from `run_oscar`. 

In [ ]:
oscar1 = local_conn.datacube_from_process(
    "run_oscar",
    oscar_endpoint="https://oscar-grnet.intertwin.fedcloud.eu",
    service="hydroform-hydromt",
    output="output",
    input_stac_url="https://stac.example.com/item/123",
    local_refresh_token="/home/jzvolensky/.local/share/openeo-python-client/refresh-tokens.json",
    oscar_script=None,
    service_config="./configs/oscar_hydromt_svc.yaml",
    context={
        "model_resolution": "0.008999999999",
        "dataset": "hydromt",
        "custom_param": "foo"
    }
)

oscar2 = local_conn.datacube_from_process(
    "run_oscar",
    oscar_endpoint="https://oscar-grnet.intertwin.fedcloud.eu",
    service="hydroform-hydromt-2",
    output="output2",
    input_stac_url=oscar1,  # <-- Pass the first node as input!
    local_refresh_token="/home/jzvolensky/.local/share/openeo-python-client/refresh-tokens.json",
    oscar_script=None,
    service_config="./configs/oscar_hydromt_svc2.yaml",
    context={
        "model_resolution": "0.0045",
        "dataset": "hydromt2",
        "custom_param": "bar"
    }
)

oscar2

## Execution

When you execute there are a couple of paths the client takes depending on the setup. In any case it will return `201` for job creation, then sleep for some time to give OSCAR a chance to setup and then we start pinging for logs which print the output stac url. 

Once we find this stac url we extract it (so make sure you keep the same format it should say `STAC OUTPUT URL` somewhere)

```py
    def parse_logs_stac(self, logs: str) -> str:
        logger.info("Parsing logs for STAC URL")
        if not logs:
            logger.error("Logs are empty, cannot parse STAC URL")
            exit(1)
        for line in logs.splitlines():
            if "STAC OUTPUT URL" in line:
                parts = line.split("STAC OUTPUT URL:")
                if len(parts) > 1:
                    stac_url = parts[1].strip()
                    logger.info(f"Found STAC OUTPUT URL: {stac_url}")
                    return stac_url
                else:
                    logger.warning("STAC OUTPUT URL line found but no URL present.")
```

This is currently not great but it works for the usecase.

Once the stac url is found we pass it to the second process which runs the exact same container and returns the output. Currently there is a lot of logging so you can see what is happening.

In [ ]:
result = oscar2.execute()
result